In [ ]:
%pip install llama-index-embeddings-huggingface llama-index-llms-openai llama-index-vector-stores-qdrant

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.core.schema import MetadataMode
from llama_index.core.readers import SimpleDirectoryReader
from qdrant_client import QdrantClient
import pandas as pd
import os 
import warnings
import re
from llama_index.llms.openai import OpenAI

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning) 

import nest_asyncio

nest_asyncio.apply()

Note: May need to copy generic_utils.py from llama_index/core/base/llms into llama_index/core/llms with current version of llama_index == version 0.10.5

In [6]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.storage import StorageContext


client = QdrantClient(location=":memory:", prefer_grpc=True,enable_hybrid=True)
vector_store = QdrantVectorStore(client=client, collection_name="lolwiki",use_async=True)
storage_context = StorageContext.from_defaults(vector_store = vector_store)


In [7]:
os.environ["OPENAI_API_KEY"] = "OPEN AI KEY"
llm = OpenAI(temperature=0.0, model="gpt-3.5-turbo")

In [ ]:
# Will have to run '%pip install llama-index-llms-huggingface' first
# Llama 2 version

#from llama_index.llms.huggingface import HuggingFaceLLM
# config= {*kwargs}
# llm = HuggingFaceLLM(
#     model="E:\BlokeLlama2\Q4\llama-2-13b-chat.Q4_K_M.gguf",
#     model_type="llama",
#     ,
#     config = config
# ))

Utilized BAAI's BGE EN Base v1.5's embeddings that were finetuned for League of Legends domain specific vocabulary that can be found [here](https://github.com/apg2275/LolWikiRAG/blob/main/LolWikiVectorEmbeddings.ipynb)

In [8]:

embeddings = HuggingFaceEmbeddings(
    model_name="PATH TO CUSTOM EMBEDS", model_kwargs={"device": "cuda"}
)

# Or can use the default model
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")


In [9]:
from llama_index.core.node_parser import HTMLNodeParser
from llama_index.core import ServiceContext
from llama_index.core import set_global_service_context

node_parser = HTMLNodeParser(chunk_size=512, chunk_overlap=26)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embeddings, node_parser=node_parser)
set_global_service_context(service_context)


In [10]:
# from llama_index.core.storage import StorageContext
from llama_index.core import VectorStoreIndex
def indexCreator(directory):
    docs = SimpleDirectoryReader(directory, filename_as_id=True).load_data()
    node_parser = HTMLNodeParser()
    docs = node_parser.get_nodes_from_documents(docs)
    vectorstore = VectorStoreIndex([], storage_context=storage_context,service_context=service_context, use_async=True, show_progress=True)
    vectorstore.insert_nodes(docs)
    return vectorstore

In [11]:
# Create the indexes for each category

champ_vector_index = indexCreator('LoLWikiHTML/Champions')
skin_vector_index = indexCreator("LoLWikiHTML/Skins")
gameplay_vector_index = indexCreator("LoLWikiHTML/GameplayElements")
item_vector_index = indexCreator("LoLWikiHTML/Items")
map_vector_index = indexCreator("LoLWikiHTML/GameModes")
patch_vector_index = indexCreator("LoLWikiHTML/PatchNotes")
voiceline_vector_index = indexCreator("LoLWikiHTML/VoiceLines")
rework_vector_index = indexCreator("LoLWikiHTML/Reworks")

In [ ]:
## Save the indexes to disk

# champ_vector_index.storage_context.persist('E:\IndexStores/Champions')
# skin_vector_index.storage_context.persist('E:\IndexStores/Skins')
# gameplay_vector_index.storage_context.persist('E:\IndexStores/GameplayElements')
# item_vector_index.storage_context.persist('E:\IndexStores/Items')
# map_vector_index.storage_context.persist('E:\IndexStores/GameModes')
# patch_vector_index.storage_context.persist('E:\IndexStores/PatchNotes')
# voiceline_vector_index.storage_context.persist('E:\IndexStores/VoiceLines')
# rework_vector_index.storage_context.persist('E:\IndexStores/Reworks')

In [60]:
from llama_index.core.query_engine.retriever_query_engine import RetrieverQueryEngine
from llama_index.core.indices.vector_store import VectorIndexRetriever
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

def retriever_and_engine(index, info):
    retriever = VectorIndexRetriever(index, vector_store_info = info,similarity_top_k=5)
    query_engine = index.as_query_engine(similarity_top_k=5, use_async=True)
    return retriever, query_engine

In [61]:
from llama_index.core.vector_stores.types import MetadataInfo, VectorStoreInfo

champ_vector_store_info = VectorStoreInfo(
    content_info="Current information about champions' abilities base stats and other gameplay information about champions",
    metadata_info=[
        MetadataInfo(
            name="Champions",
            type="str",
            description="Collection of champions",
        )
    ]
)


skin_vector_store_info = VectorStoreInfo(
    content_info="Current information about champion's skins and other cosmetic information relating to the champions/characters in League of Legends",
    metadata_info=[
        MetadataInfo(
            name="Skins",
            type="str",
            description="Skin and Cosmetic information for champions in League of Legends",
        )
    ]
)

gameplay_vector_store_info = VectorStoreInfo(
    content_info="Information about the gameplay mechanics and interactions such as ability types and jungle within the game League of Legends",
    metadata_info=[
        MetadataInfo(
            name="GameplayMechanics",
            type="str",
            description="Information about what the ability properties are and how the game works in general.",
        )
    ]
)

item_vector_store_info = VectorStoreInfo(
    content_info="Information about items and their properities/stats in League of Legends",
    metadata_info=[
        MetadataInfo(
            name="Items",
            type="str",
            description="Items and their properties in League of Legends.",
        )
    ]
)

map_vector_store_info = VectorStoreInfo(
    content_info="Information maps and game modes in League of Legends",
    metadata_info=[
        MetadataInfo(
            name="GameModes",
            type="str",
            description="Information about maps/game modes in League of Legends.",
        )
    ]
)

patch_vector_store_info = VectorStoreInfo(
    content_info="Historical information about patches AKA changes to the game League of Legends",
    metadata_info=[
        MetadataInfo(
            name="Patches",
            type="str",
            description="Patch notes and changes to the game League of Legends.",
        )
    ]
)

voiceline_vector_store_info = VectorStoreInfo(
    content_info="Information voice lines and audio information in League of Legends",
    metadata_info=[
        MetadataInfo(
            name="VoiceLines",
            type="str",
            description="Voice lines and speech from champions.",
        )
    ]
)

rework_vector_store_info = VectorStoreInfo(
    content_info="Historical information about large scale changes and removed or reworked abilities in League of Legends that are no longer in the game",
    metadata_info=[
        MetadataInfo(
            name="Reworks",
            type="str",
            description="Information about abilities that have been reworked or removed from the game.",
        )
    ]
)

In [62]:
champ_vector_auto_retriever, champ_query_engine = retriever_and_engine(champ_vector_index, champ_vector_store_info)
skin_vector_auto_retriever, skin_query_engine = retriever_and_engine(skin_vector_index, skin_vector_store_info)
gameplay_vector_auto_retriever, gameplay_query_engine = retriever_and_engine(gameplay_vector_index, gameplay_vector_store_info)
item_vector_auto_retriever, item_query_engine = retriever_and_engine(item_vector_index, item_vector_store_info)
map_vector_auto_retriever, map_query_engine = retriever_and_engine(map_vector_index, map_vector_store_info)
patch_vector_auto_retriever, patch_query_engine = retriever_and_engine(patch_vector_index, patch_vector_store_info)
voiceline_vector_auto_retriever, voiceline_query_engine = retriever_and_engine(voiceline_vector_index, voiceline_vector_store_info)
rework_vector_auto_retriever, rework_query_engine = retriever_and_engine(rework_vector_index, rework_vector_store_info)

In [64]:
from llama_index.core.tools import RetrieverTool

retriever_tools = [
RetrieverTool.from_defaults(
    retriever= champ_vector_auto_retriever,
    name="Champions",
    description="Provides current information about champion's gameplay information such as: abilities and their properties, stats and map specific changes.",
),
RetrieverTool.from_defaults(
    retriever= skin_vector_auto_retriever,
    name="Skins",
    description=
        "Provides information about skins and other cosmetic information for champions."
    )
,
RetrieverTool.from_defaults(
    retriever=item_vector_auto_retriever,
    name="Items",
    description=
        "Provides information about items and their stats and properties."
    )
,
RetrieverTool.from_defaults(
    retriever=map_vector_auto_retriever,
    name = "GameModes",
    description="Provides information about maps and game modes in League of Legends."
    )
,
RetrieverTool.from_defaults(
    retriever=patch_vector_auto_retriever,
    name="Patches",
    description=
        "Provides information about patches and changes to the game."
    )
,
RetrieverTool.from_defaults(
    retriever=voiceline_vector_auto_retriever,
    name="VoiceLines",
    description=
        "Provides information about voice lines, speech, and audio in League of Legends."
    )
,
RetrieverTool.from_defaults(
    retriever=rework_vector_auto_retriever,
    name="Reworks",
    description=
        "Provides old information about large scale changes and removed and/or reworked abilities in League of Legends that are no longer in the game."
    )
]

Table of champion's base stats taken from [LolWiki's Champion Data Table](https://leagueoflegends.fandom.com/wiki/Module:ChampionData/data?action=edit). Then converetd the lua format to an dict using the package luadata then from that converted to csv.

In [ ]:
# df = pd.read_csv("LolWikiHTML\champion_stat_dict.csv")

In [ ]:
# from llama_index.core.query_engine import PandasQueryEngine
# pandas_engine = PandasQueryEngine(df, verbose=True)

In [ ]:
# QueryEngineTool.from_defaults(
#     query_engine= pandas_engine,
#     name="Pandas Query",
#     description=
#         "Useful for turning natural language into a pandas query for information about champion's base stats."
#     )

In [65]:
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine import SubQuestionQueryEngine
query_engine_tools = [
QueryEngineTool.from_defaults(
    query_engine= champ_query_engine,
    name="Champions",
    description="Provides current information about specific champions' gameplay information such as: abilities and their properties, stats and map/game mode specific changes",
),
QueryEngineTool.from_defaults(
    query_engine= skin_query_engine,
    name="Skins",
    description=
        "Provides information about skins and other cosmetic information for champions."
    )
,
QueryEngineTool.from_defaults(
    query_engine= gameplay_query_engine,
    name="GameplayMechanics",
    description=
        "Provides information about the gameplay mechanics and interactions such as ability types and jungle within the game League of Legends."
    )
,
QueryEngineTool.from_defaults(
    query_engine=item_query_engine,
    name="Items",
    description=
        "Provides information about items and their stats and properties."
    )
,
QueryEngineTool.from_defaults(
    query_engine= map_query_engine,
    name="GameModes",
    description=
        "Provides information about maps and game modes in League of Legends."
    )
,
QueryEngineTool.from_defaults(
    query_engine= patch_query_engine,
    name="Patches",
    description=
        "Provides information about patches and changes to the game."
    )
,
QueryEngineTool.from_defaults(
    query_engine= voiceline_query_engine,
    name="VoiceLines",
    description=
        "Provides information about voice lines, speech, and audio in League of Legends."
    )
,
QueryEngineTool.from_defaults(
    query_engine= rework_query_engine,
    name="Reworks",
    description=
        "Provides old information about large scale changes and removed and/or reworked abilities in League of Legends that are no longer in the game."
    )


]

In [77]:
from llama_index.core.prompts import PromptTemplate

template = """"\
### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

Your goal is to provide answers relating a video game. 
 Be sure to think carefully and follow through with logic and think step by step when thinking about interactions between champions, items, abilities, etc. 
 When prompted for information be sure to relay the exact values.

 If an answer contains several parts be sure to list all of them out. For example, if asked about how many skins a champion has, list out the names of *ALL* the skins.

 Based on the provided context, please answer the provided question. You can only use the provided context to answer the question.
If you do not know the answer - please respond with "I don't know".

Above all else make sure your answer is accurate.

Question:
{question}

### Response:
"""

rag_prompt = PromptTemplate(template)

In [78]:
from llama_index.core.retrievers import RouterRetriever
from llama_index.core.selectors.pydantic_selectors import PydanticMultiSelector

multiretrieve = RouterRetriever(
    selector=PydanticMultiSelector.from_defaults(),
    retriever_tools = retriever_tools,
)

In [95]:
from llama_index.core.query_pipeline.query import QueryPipeline
from llama_index.core.postprocessor import LLMRerank
from llama_index.core.response_synthesizers import get_response_synthesizer
reranker = LLMRerank(top_n=4)
summarizer = get_response_synthesizer(
    response_mode="refine",
    llm=llm,
    verbose=True,
    structured_answer_filtering=True
)


p = QueryPipeline(verbose=True)
p.add_modules(
    {
        "llm": llm,
        "prompt_tmpl": rag_prompt,
        "retriever": multiretrieve,
        "reranker": reranker,
        "summary": summarizer
    }
)
p.add_link("prompt_tmpl", "llm")
p.add_link("llm", "retriever")
p.add_link("retriever", "reranker", dest_key="nodes")
p.add_link("llm", "reranker", dest_key="query_str")
p.add_link("reranker", "summary", dest_key="nodes")
p.add_link("llm", "summary", dest_key='query_str')

In [81]:
output = p.run(question="Does Olaf lose his bonus armor and mr after ulting?")

> Running module prompt_tmpl with input: 
question: Does Olaf lose his bonus armor and mr after ulting?

> Running module llm with input: 
messages: "### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

Your goal is to provide answers relating a video game. 
 Be sure to think c...

> Running module retriever with input: 
input: assistant: No, Olaf does not lose his bonus armor and magic resist after ulting. His ultimate ability, Ragnarok, grants him bonus attack damage, immunity to crowd control effects, and increased attack...

> Running module reranker with input: 
query_str: assistant: No, Olaf does not lose his bonus armor and magic resist after ulting. His ultimate ability, Ragnarok, grants him bonus attack damage, immunity to crowd control effects, and increased attack...
nodes: [NodeWithScore(node=TextNode(id_='6980992f-7869-4e03-90ad-470336cf65e2', embedding=None, metadata={'tag': 'li', 'file_path': 'E:\\LoL

In [82]:
print(output)

False, Olaf no longer loses his bonus resistances upon casting his ultimate ability, Ragnarok. The bonus resistances are now retained after using Ragnarok.


In [85]:
output2 = p.run(question="Does Gragas E stop Lee sin Q?")

> Running module prompt_tmpl with input: 
question: Does Gragas E stop Lee sin Q?

> Running module llm with input: 
messages: "### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

Your goal is to provide answers relating a video game. 
 Be sure to think c...

> Running module retriever with input: 
input: assistant: Yes, Gragas' E ability (Body Slam) can stop Lee Sin's Q ability (Sonic Wave/Resonating Strike). When Gragas uses Body Slam, he dashes to a target location and collides with the first enemy ...

> Running module reranker with input: 
query_str: assistant: Yes, Gragas' E ability (Body Slam) can stop Lee Sin's Q ability (Sonic Wave/Resonating Strike). When Gragas uses Body Slam, he dashes to a target location and collides with the first enemy ...
nodes: [NodeWithScore(node=TextNode(id_='3199bfc3-64e6-4bb9-a61d-51d6182408d3', embedding=None, metadata={'tag': 'li', 'file_path': 'E:\\LoLWikiHTML\\Champions\\G

In [88]:
output2.response

"Yes, Gragas' E ability (Body Slam) can interrupt Lee Sin's Q ability (Sonic Wave/Resonating Strike). When Gragas uses Body Slam, he dashes to a target location and collides with the first enemy unit he hits, knocking them back and dealing damage. If Gragas hits Lee Sin with Body Slam while Lee Sin is using Sonic Wave (Q), it will interrupt Lee Sin's dash and prevent him from following up with Resonating Strike."

In [ ]:
output3 = p.run(question="What is Akali's kit?")

> Running module prompt_tmpl with input: 
question: What is Akali's kit?

> Running module llm with input: 
messages: "### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

Your goal is to provide answers relating a video game. 
 Be sure to think c...

> Running module retriever with input: 
input: assistant: Akali's kit consists of the following abilities:

1. Passive - Assassin's Mark: Akali's first two basic attacks against an enemy champion will deal bonus magic damage and restore energy.
2....

> Running module reranker with input: 
query_str: assistant: Akali's kit consists of the following abilities:

1. Passive - Assassin's Mark: Akali's first two basic attacks against an enemy champion will deal bonus magic damage and restore energy.
2....
nodes: [NodeWithScore(node=TextNode(id_='65c9d8ab-e1bf-4d49-96a2-eb738ff66148', embedding=None, metadata={'tag': 'li', 'file_path': 'E:\\LoLWikiHTML\\Champions\\Akali_LoL.

In [90]:
print(output3)

Akali's abilities include Passive - Assassin's Mark, Q - Five Point Strike, W - Twilight Shroud, E - Shuriken Flip, R - Perfect Execution.


In [91]:
output4 = p.run(question="Does Yorick's ghouls proc Liandry's?")

> Running module prompt_tmpl with input: 
question: Does Yorick's ghouls proc Liandry's?

> Running module llm with input: 
messages: "### Instruction:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

Your goal is to provide answers relating a video game. 
 Be sure to think c...

> Running module retriever with input: 
input: assistant: Yes, Yorick's ghouls do proc Liandry's Torment. Liandry's Torment deals bonus damage over time based on the target's current health, and Yorick's ghouls' attacks count as single target spel...

> Running module reranker with input: 
query_str: assistant: Yes, Yorick's ghouls do proc Liandry's Torment. Liandry's Torment deals bonus damage over time based on the target's current health, and Yorick's ghouls' attacks count as single target spel...
nodes: [NodeWithScore(node=TextNode(id_='f3d89ee0-8ca8-4c81-b7e2-f26bcc52437a', embedding=None, metadata={'tag': 'li', 'file_path': 'E:\\LoLWikiHTML\\Items

In [92]:
print(output4)

Yes, Yorick's ghouls can trigger Liandry's Torment. The burn effect from Liandry's Torment, which deals magic damage based on the target's maximum health, is applied when Yorick's ghouls deal ability damage, as their attacks count as single target spells.
